In [1]:
import logging

from pydub import AudioSegment

import pandas as pd

AudioSegment.ffmpeg = "C:\\installed\\ffmpeg\\bin\\ffmpeg.exe"
AudioSegment.converter = "C:\\installed\\ffmpeg\\bin\\ffmpeg.exe"

from watson_developer_cloud import TextToSpeechV1

import glob
import json
from random import shuffle

from datetime import datetime

import os
import numpy as np

import csv

# Rohun's IBM credential
tts = TextToSpeechV1(
    username='683a066f-a063-4e2e-bcae-1c48690bd515',
    password='7oPU2XALqUtv',
)
def synthesize_number(number):
    try:
        byte_audio = tts.synthesize(str(number), accept='audio/wav', voice="en-US_AllisonVoice")
    except Exception as e:
        try:
            byte_audio = tts.synthesize(str(number), accept='audio/wav', voice="en-US_AllisonVoice")
        except Exception as e:
            return AudioSegment.silent(duration=250)

    # The sample_width, frame_rate and the channels are empirical values. No values in documentation.
    sound = AudioSegment(
        data=byte_audio,

        sample_width=2,
        frame_rate=22050,
        channels=1
    )

    return sound


def stich_audio_files(file_list, silence, post_number_silence, location, tag_extension):
    stiched_result = AudioSegment.empty()

    USER_STUDY_OUTPUT_DATA = "user_study_output\\"
    for index, audio_file in enumerate(file_list):
        try:
            audio = AudioSegment.from_file(USER_STUDY_OUTPUT_DATA + location + audio_file + tag_extension, format="wav")
            number = synthesize_number(index + 1)
            stiched_result += silence + number + post_number_silence + audio
        except Exception as e:
            logging.error(str(e))

    return stiched_result



def crop_and_save_to_wav(file_name):
    just_name = file_name.rfind(".")
    audio = AudioSegment.from_file(file_name, format="mp3")[3000:6000]
    
    for value in [0, 10, 20, 30, 40]:
        reduced_audio = audio - value
        reduced_audio.export(file_name[:just_name] + "_" + str(value) + ".wav", format="wav")


        
def decrease_beep_strength():
    beep = AudioSegment.from_file("data_input\\" + 'beep.wav', 'wav')
    for i in range(5):
        beep = beep - 10
        beep.export(out_f="data_input\\beep_" + str(i) + ".wav", format="wav")
        

    
'Reads files with the specified Noise types in allowed values'

def read_files(file_list, allowed_values, columns):
    result_dataframe = pd.DataFrame()
    
    for file_name in file_list:
        data_frame = pd.read_csv(file_name, names=columns, usecols=range(len(columns)))
        if len(data_frame) == 0:
            continue
        
        result_dataframe = result_dataframe.append(data_frame, ignore_index=True)
    
    result_dataframe = result_dataframe[result_dataframe["noise_type"].isin(allowed_values)]
    
    result_dataframe = result_dataframe[result_dataframe["noise"] != -1]
    
    return result_dataframe

In [2]:
def stich_audio_alternate_files():
    """Stiches 5 second random char Google CAPTCHAs to create 10 second CAPTCHAs"""
    
    items = json.loads('''[ {"audio":"output_149449394454942.wav", "gt":"89016"},
  {"audio":"output_149449657320933.wav", "gt":"43417"},
  {"audio":"output_1494487771961015.wav", "gt":"81330"},
  {"audio":"output_1494493574786677.wav", "gt":"62384"},
  {"audio":"output_1494496272313691.wav", "gt":"73592"},
  {"audio":"output_1494496294872982.wav", "gt":"10105"},
  {"audio":"output_1494496519281245.wav", "gt":"38695"},
  {"audio":"output_1494496601054923.wav", "gt":"80168"},
  {"audio":"output_1494496664266538.wav", "gt":"85780"},
  {"audio":"output_1494497754558948.wav", "gt":"99358"},
  {"audio":"output_1494497804519805.wav", "gt":"08884"},
  {"audio":"output_1494497859133929.wav", "gt":"47232"},
  {"audio":"output_1494497882023238.wav", "gt":"93916"},
  {"audio":"output_1494497934197222.wav", "gt":"24001"}]''') 
    
    list1 = []
    
    for first, second in zip(items[0:7], items[7:]):    
        stiched_result = AudioSegment.empty()

        c1_source_data = "Test_Data\\v1\\c1\\Google_Captcha_Demo_5_CHAR\\"
        
        c1_output_data = "Test_Data\\v3_2\\c1\\"
        
        audio = AudioSegment.from_file(c1_source_data + first["audio"], format="wav")
        stiched_result += audio
        audio = AudioSegment.from_file(c1_source_data + second["audio"], format="wav")
        stiched_result += audio
        
        gt = first["gt"] + second["gt"]
        
        list1.append({"audio" : first["audio"], "gt" : gt})
        
        
        # stiched_result.export(c1_output_data + first["audio"], "wav")
    print(str(list1))

stich_audio_alternate_files()

[{'audio': 'output_149449394454942.wav', 'gt': '8901680168'}, {'audio': 'output_149449657320933.wav', 'gt': '4341785780'}, {'audio': 'output_1494487771961015.wav', 'gt': '8133099358'}, {'audio': 'output_1494493574786677.wav', 'gt': '6238408884'}, {'audio': 'output_1494496272313691.wav', 'gt': '7359247232'}, {'audio': 'output_1494496294872982.wav', 'gt': '1010593916'}, {'audio': 'output_1494496519281245.wav', 'gt': '3869524001'}]


In [37]:
'''Parameters that are being used in this exercise'''

# Data Set Iteration number
data_version = "3b"

# Captcha Version. Options - 2, 3a, 3b, 4
captcha_type = "4"

# filter_keyword_list = ["REFACTORED_White_YT_VERSION_" + captcha_type, "REFACTORED_White_PODCAST_VERSION_" + captcha_type, "REFACTORED_PODCAST_VERSION_" + captcha_type]
filter_keyword_list = ["REDONE_REFACTORED_PODCAST_VERSION_" + captcha_type]

In [38]:
# add tag and time to name and use json to dump file names. json file uses same name.

complete_file_list = []
for filter_keyword in filter_keyword_list:
    complete_file_list.extend(glob.glob("logs\\*selected_*" + filter_keyword + ".csv"))

if captcha_type == "4":
    column_word = ["name", "version", "original_text", "noise", "complete", "source_type", "noise_type", "transcript", "reduced_word"]
else:
    column_word = ["name", "version", "start", "end", "original_text", "noise", "first_word_easy", "first_confidence", "second_confidence", "source_type", "noise_type", "first_word", "second_word"]

file_dataframe= read_files(complete_file_list, ["White"], column_word)

In [6]:
def create_filtered_aname_file(file_text, data_version, captcha_type):
    'Copies the N files selected and stored in the log file from the specified source location to "vX" destination folder'
        

    audio_name_list = []
   
    for audio_file in file_text.split("\n"):
        audio_name_list.append({"audio" : audio_file.strip()})

    audio_folder = "Test_data\\v" + data_version + "\\audioname"
    os.makedirs(audio_folder, exist_ok = True)
    json.dump(audio_name_list, open(os.path.join(audio_folder, "aname" + captcha_type + ".json"), "w"))
    
    
create_filtered_aname_file('''Mcluhan-Mckenna_1_chunk_27_1499080254532628_count_1_noise_26_noise_type_White.wav
Shakespeare_chunk_60_1497156637776667_count_0_noise_10_noise_type_White.wav
TerenceMckenna-TrueHallucinations09-16_chunk_64_1499080254532628_count_0_noise_7_noise_type_White.wav
American_Civil_War_chunk_54_1497156637776667_count_0_noise_28_noise_type_White.wav
American_revolution_lecture_chunk_47_1497156637776667_count_3_noise_15_noise_type_White.wav
Darwins_Legacy_chunk_151_1497156637776667_count_0_noise_27_noise_type_White.wav
History_4A_Fall_2007_UC_Berkeley_Lecture_24_Monarchy_at_Rome_The_Age_of_Augustus_20476_chunk_111_1499080254532628_count_0_noise_0_noise_type_White.wav
History_4A_Fall_2007_UC_Berkeley_Lecture_24_Monarchy_at_Rome_The_Age_of_Augustus_20476_chunk_129_1499080254532628_count_0_noise_0_noise_type_White.wav
Nikola_Tesla_chunk_30_1497156637776667_count_1_noise_7_noise_type_White.wav
Shakespeare_chunk_38_1497156637776667_count_0_noise_23_noise_type_White.wav
TerenceMckenna-TrueHallucinations08-16_chunk_21_1499080254532628_count_0_noise_0_noise_type_White.wav
TerenceMckenna-TrueHallucinations08-16_chunk_82_1499080254532628_count_0_noise_6_noise_type_White.wav
TerenceMckenna-TrueHallucinations12-16_chunk_64_1499080254532628_count_0_noise_18_noise_type_White.wav
Yoshua_Bengio_chunk_59_1497156637776667_count_0_noise_4_noise_type_White.wav
Google_IO_2017_chunk_32_1497156637776667_count_0_noise_23_noise_type_White.wav
Nikola_Tesla_chunk_15_1497534345207332_count_0_noise_22_noise_type_White.wav
TerenceMckenna-TrueHallucinations02-16_chunk_58_1499080254532628_count_0_noise_14_noise_type_White.wav
TerenceMckenna-TrueHallucinations06-16_chunk_97_1499080254532628_count_0_noise_13_noise_type_White.wav
TerenceMckenna-TrueHallucinations08-16_chunk_3_1499080254532628_count_1_noise_17_noise_type_White.wav
TerenceMckenna-TrueHallucinations08-16_chunk_41_1499080254532628_count_0_noise_9_noise_type_White.wav
TheVoynichManuscript_chunk_117_1499080254532628_count_1_noise_0_noise_type_White.wav
Western_Philosophy_chunk_58_1497534345207332_count_0_noise_15_noise_type_White.wav
Yoshua_Bengio_chunk_59_1497156637776667_count_1_noise_0_noise_type_White.wav
American_revolution_lecture_chunk_107_1497534345207332_count_2_noise_23_noise_type_White.wav
TerenceMckenna-TrueHallucinations04-16_chunk_57_1499080254532628_count_0_noise_25_noise_type_White.wav
TerenceMckenna-TrueHallucinations08-16_chunk_82_1499080254532628_count_1_noise_26_noise_type_White.wav
TerenceMckenna-TrueHallucinations09-16_chunk_63_1499080254532628_count_0_noise_11_noise_type_White.wav
TerenceMckenna-TrueHallucinations10-16_chunk_14_1499080254532628_count_1_noise_19_noise_type_White.wav
FinneginsWake_chunk_129_1499080254532628_count_0_noise_7_noise_type_White.wav
google_Documentary_chunk_60_1497156637776667_count_0_noise_9_noise_type_White.wav
Google_IO_2017_chunk_196_1497534345207332_count_1_noise_19_noise_type_White.wav
Google_IO_2017_chunk_96_1497156637776667_count_0_noise_18_noise_type_White.wav
TerenceMckenna-TrueHallucinations06-16_chunk_63_1499080254532628_count_0_noise_21_noise_type_White.wav
TerenceMckenna-TrueHallucinations10-16_chunk_14_1499080254532628_count_0_noise_13_noise_type_White.wav
TerenceMckenna-TrueHallucinations12-16_chunk_63_1499080254532628_count_0_noise_29_noise_type_White.wav
TerenceMckenna-TrueHallucinations15-16_chunk_46_1499080254532628_count_0_noise_31_noise_type_White.wav''', "3_3", "4")    
    
def create_ten(file_dataframe, source, data_version, captcha_type):
    'Copies the N files selected and stored in the log file from the specified source location to "vX" destination folder'
    
    NUMBER_TO_SELECT = 1000
    
    destination = "Test_data\\v" + data_version + "\\c" + captcha_type + "_" + str(datetime.now().timestamp()).replace(".","")
    os.makedirs(destination, exist_ok = True)

    dictionary_gt = []
    audio_name_list = []
    rows = [["Name", "Captcha", "Word_Detected", "Text_Detected"]]
   
    for index, row in file_dataframe.head(NUMBER_TO_SELECT).iterrows():
        audio_file = row["name"]    
        
        if captcha_type == "4":
            transcript = row["reduced_word"]
        else: 
            if row["first_word_easy"]:
                transcript = row["first_word"]
            else:
                transcript = row["second_word"]
        
        dictionary_gt.append({"audio" : audio_file + ".wav", "gt" : transcript})
        audio_name_list.append({"audio" : audio_file + ".wav"})
        rows.append([audio_file, captcha_type])
        
        audio = AudioSegment.from_file(source + audio_file + ".wav", format="wav")
        output_path = os.path.join(destination, audio_file + ".wav")
        audio.export(output_path, format="wav")
        
    gt_folder = "Test_data\\v" + data_version + "\\gt_data"
    os.makedirs(gt_folder, exist_ok = True)
    json.dump(dictionary_gt, open(os.path.join(gt_folder, "gt" + captcha_type + ".json"), "w"))
    
    audio_folder = "Test_data\\v" + data_version + "\\audioname"
    os.makedirs(audio_folder, exist_ok = True)
    json.dump(audio_name_list, open(os.path.join(audio_folder, "aname" + captcha_type + ".json"), "w"))
    
    selection_folder = "Test_data\\v" + data_version + "\\selection"
    os.makedirs(selection_folder, exist_ok = True)
    with open(os.path.join(selection_folder, "sample" + captcha_type + ".csv"), "w", newline="") as sample_file:
        csv.writer(sample_file).writerows(rows)

In [40]:
file_dataframe

,name,version,original_text,noise,complete,source_type,noise_type,transcript,reduced_word
2,History_4A_Fall_2007_UC_Berkeley_Lecture_27_St...,4,perimeter was reinforced by these artificial d...,0,"{'length': 9, 'confidence': 0.7673, 'end_time'...",podcast_lecture,White,"[{'timestamps': [['for', 0.32, 0.43], ['him', ...",perimeter
6,TerenceMckenna-TrueHallucinations07-16_chunk_5...,4,approaching break through it says to me,18,"{'length': 11, 'confidence': 0.9998, 'end_time...",podcast_lecture,White,"[{'timestamps': [['approaching', 0.32, 0.89], ...",through
7,OpenDoors_chunk_131_1499227381638939_count_0_n...,4,transforming machine now that seem to do,12,"{'length': 12, 'confidence': 0.9998, 'end_time...",podcast_lecture,White,"[{'timestamps': [['transforming', 0.28, 1.22],...",that
10,DennisMcKenna5_chunk_19_1499227381638939_count...,4,only held off lady that I signed,0,"{'length': 4, 'confidence': 0.3752, 'end_time'...",podcast_lecture,White,"[{'timestamps': [['I', 0.23, 0.27], ['only', 0...",signed
15,History_4A_Fall_2007_UC_Berkeley_Lecture_28_Cr...,4,was captured by the fast king shop,8,"{'length': 3, 'confidence': 0.9844, 'end_time'...",podcast_lecture,White,"[{'timestamps': [['was', 0.29, 0.41], ['captur...",king
19,FinneginsWake_chunk_168_1499227381638939_count...,4,not our apartments then we're no better,29,"{'length': 3, 'confidence': 0.4126, 'end_time'...",podcast_lecture,White,"[{'timestamps': [['our', 0.3, 0.54], ['apartme...",then
20,BeyondPsychology_chunk_91_1499259869456154_cou...,4,a visible glow and the next year,26,"{'confidence': 0.6442, 'start_time': 1.8, 'wor...",podcast_lecture,White,"[{'transcript': 'video ', 'word_confidence': [...",next
22,FinneginsWake_chunk_127_1499259869456154_count...,4,tension within her cares as it says,19,"{'confidence': 0.5761, 'start_time': 2.59, 'wo...",podcast_lecture,White,[{'transcript': 'tension offend her cares as i...,cares
23,History_4A_Fall_2007_UC_Berkeley_Lecture_04_Hi...,4,a time drought but that route was,9,"{'confidence': 0.9435, 'start_time': 17.65, 'w...",podcast_lecture,White,[{'transcript': 'this time around but that tha...,drought
24,History_4A_Fall_2007_UC_Berkeley_Lecture_04_Hi...,4,that route was when you could harvest,9,"{'confidence': 0.8613, 'start_time': 19.89, 'w...",podcast_lecture,White,[{'transcript': 'that that was when you could ...,harvest


In [41]:
create_ten(file_dataframe, "C:\\Users\\IBM_ADMIN\\speech_recognition\\data_output_selected\\", data_version, captcha_type)

In [32]:
complete_file_list = glob.glob("user_study_output\\user_study_initial_output\\*.wav")
shuffle(complete_file_list)

dbfs_list = []

In [33]:
for file_entry in complete_file_list:
    audio = AudioSegment.from_file(file_entry, format="wav")
    dbfs_list.append(audio.dBFS)

In [47]:
mean = np.mean(dbfs_list)
std = np.std(dbfs_list)

print(mean)
print(std)

number = 0
for dbfs_value in dbfs_list:
    number += 1 if dbfs_value > (mean + std) else 0
#     if dbfs_value > (mean + std):
#         print(dbfs_value)
print((mean + std))
print(number)

-15.930842618
4.49456667099
-11.436275947
169


In [12]:
'uses selected_file_data for creating a stiched audio file'

def create_stiched_output(selected_file_data, output_tag):  
    output_file_prefix = "stiched_output\\study" + output_tag

    json.dump(selected_file_data, open(output_file_prefix + ".json", "w"))

    silence = AudioSegment.silent(duration=1000)
    post_number_silence = AudioSegment.silent(duration=500)

    source_location = "reduced_confidence\\"

    for tagged_format in ["_W_B.wav", "_O_B.wav", ".wav"]:

        f = stich_audio_files(selected_file_data, silence, post_number_silence, source_location, tagged_format)

        f.export(out_f="stiched_output\\study" + output_tag + tagged_format, format="wav")

        print("Done for {}", tagged_format)

In [35]:
# Bin the video accordoing to the initial confidence of the strong word.

def add_to_appropriate_df(original, variable_set):
    for df_dict in variable_set:

        first_criteria = (original.first_word_easy == True) & (original.first_confidence > df_dict["low"]) & (original.first_confidence <= df_dict["high"])
        second_criteria = (original.first_word_easy == False) & (original.second_confidence > df_dict["low"]) & (original.second_confidence <= df_dict["high"])
        
        df_dict["df"] = df_dict["df"].append(original[first_criteria])
        df_dict["df"] = df_dict["df"].append(original[second_criteria])
    
    return variable_set


variable_set = []

step = 5
for counter in range(70, 100, step):
    variable_set.append({"low" : float(counter)/100, "high" : float(counter + step)/100, "df" : pd.DataFrame()})
    
variable_set = add_to_appropriate_df(file_dataframe, variable_set)

output_tag = "_Two_Words_" + str(datetime.now().timestamp()).replace(".","")

def create_audio_by_confidence():

    for df_dict in variable_set:
        df = df_dict["df"]
        if len(df) == 0:
            continue

        extended_tag = output_tag + "_Low_" + str(df_dict["low"]) + "_High_" + str(df_dict["high"])
        selected_file_list = list(df.name)[:Number_of_entries]

        create_stiched_output(selected_file_list, extended_tag)

        print("Done for : ", extended_tag)

def create_for_complete_list():
    # shuffle(complete_file_data)
    create_stiched_output(list(file_dataframe.name), output_tag)
    
create_for_complete_list()